# Threading (потоки)

In [6]:
import logging
import threading
import time


def thread_fun(name):
    logging.info(f"Thread {name}: starting")
    time.sleep(2)
    logging.info(f"Thread {name}: finishing")

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

    logging.info("Main : before creating thread")

    logging.info("Main : before running thread")

    thread_fun(1)
    thread_fun(2)
    thread_fun(3)

    logging.info("Main : wait for thread to finish")

    logging.info("Main : all done")

17:22:48: Main : before creating thread
17:22:48: Main : before running thread
17:22:48: Thread 1: starting
17:22:50: Thread 1: finishing
17:22:50: Thread 2: starting
17:22:52: Thread 2: finishing
17:22:52: Thread 3: starting
17:22:54: Thread 3: finishing
17:22:54: Main : wait for thread to finish
17:22:54: Main : all done


In [4]:
import logging
import threading
import time


def thread_fun(name):
    logging.info(f"Thread {name}: starting")
    time.sleep(2)
    logging.info(f"Thread {name}: finishing")

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

    logging.info("Main : before creating thread")

    x = threading.Thread(target=thread_fun, args=(1,))

    logging.info("Main : before running thread")

    x.start()

    logging.info("Main : wait for thread to finish")

    logging.info("Main : all done")

17:08:06: Main : before creating thread
17:08:06: Main : before running thread
17:08:06: Thread 1: starting
17:08:06: Main : wait for thread to finish
17:08:06: Main : all done


17:08:08: Thread 1: finishing


In [7]:
import logging
import threading
import time


def thread_fun(name):
    logging.info(f"Thread {name}: starting")
    time.sleep(2)
    logging.info(f"Thread {name}: finishing")

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

    logging.info("Main : before creating thread")

    x = threading.Thread(target=thread_fun, args=(1,))

    logging.info("Main : before running thread")

    x.start()

    logging.info("Main : wait for thread to finish")
    x.join()
    
    logging.info("Main : all done")

17:24:23: Main : before creating thread
17:24:23: Main : before running thread
17:24:24: Thread 1: starting
17:24:24: Main : wait for thread to finish
17:24:26: Thread 1: finishing
17:24:26: Main : all done


In [13]:
import logging
import threading
import time


def thread_fun(name):
    logging.info(f"Thread {name}: starting")
    time.sleep(2)
    logging.info(f"Thread {name}: finishing")

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")


    threads = []
    for i in range(3):
        logging.info(f"Main : create and run thread {i}")
        x = threading.Thread(target=thread_fun, args=(i,))
        threads.append(x)
        x.start()

    logging.info("-------------------------")
    
    for i, t in enumerate(threads):
        logging.info(f"Main : wait for thread {i} to finish")
        t.join()
        logging.info(f"Main : thread {i} done")

17:35:49: Main : create and run thread 0
17:35:49: Thread 0: starting
17:35:49: Main : create and run thread 1
17:35:49: Thread 1: starting
17:35:49: Main : create and run thread 2
17:35:49: Thread 2: starting
17:35:49: -------------------------
17:35:49: Main : wait for thread 0 to finish
17:35:51: Thread 0: finishing
17:35:51: Main : thread 0 done
17:35:51: Thread 1: finishing
17:35:51: Main : wait for thread 1 to finish
17:35:51: Main : thread 1 done
17:35:51: Thread 2: finishing
17:35:51: Main : wait for thread 2 to finish
17:35:51: Main : thread 2 done


In [22]:
import logging
import threading
import time
from concurrent.futures import ThreadPoolExecutor


def thread_fun(name):
    logging.info(f"Thread {name}: starting")
    time.sleep(2)
    logging.info(f"Thread {name}: finishing")

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")


    with ThreadPoolExecutor(max_workers=3) as executor:
        executor.map(thread_fun, ['John', 'Jane', 'Jimmy', 'Josh', 'Jeremy'])

17:45:39: Thread John: starting
17:45:39: Thread Jane: starting
17:45:39: Thread Jimmy: starting
17:45:41: Thread John: finishing
17:45:41: Thread Josh: starting
17:45:41: Thread Jane: finishing
17:45:41: Thread Jimmy: finishing
17:45:41: Thread Jeremy: starting
17:45:43: Thread Josh: finishing
17:45:43: Thread Jeremy: finishing


## Race conditions (состояние гонки)

In [25]:
class FakeDatabase:
    def __init__(self):
        self.value = 0

    def update(self, name):
        logging.info(f"Thread {name}: starting update")
        local_copy = self.value
        local_copy += 1
        time.sleep(0.1)
        self.value = local_copy
        logging.info(f"Thread {name}: finishing update")


if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

    db = FakeDatabase()
    logging.info(f"Testing update. Starting value is {db.value}")

    with ThreadPoolExecutor(max_workers=2) as executor:
        for i in range(2):
            executor.submit(db.update, i)
            
    logging.info(f"Testing update. Ending value is {db.value}")


17:56:29: Testing update. Starting value is 0
17:56:29: Thread 0: starting update
17:56:29: Thread 1: starting update
17:56:29: Thread 0: finishing update
17:56:29: Thread 1: finishing update
17:56:29: Testing update. Ending value is 1


In [32]:
class FakeDatabase:
    def __init__(self):
        self.value = 0
        self._lock = threading.Lock()

    def locked_update(self, name):
        logging.info(f"Thread {name}: starting update")
        logging.info(f"Thread {name} about to lock")
        with self._lock:
            logging.info(f"Thread {name} has lock")
            local_copy = self.value
            local_copy += 1
            time.sleep(0.1)
            self.value = local_copy
            logging.info(f"Thread {name} about to release lock")
        
        logging.info(f"Thread {name} after release")
        logging.info(f"Thread {name}: finishing update")


if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

    db = FakeDatabase()
    logging.info(f"Testing update. Starting value is {db.value}")

    with ThreadPoolExecutor(max_workers=2) as executor:
        for i in range(2):
            executor.submit(db.locked_update, i)
            
    logging.info(f"Testing update. Ending value is {db.value}")


18:01:31: Testing update. Starting value is 0
18:01:31: Thread 0: starting update
18:01:31: Thread 1: starting update
18:01:31: Thread 1 about to lock
18:01:31: Thread 1 has lock
18:01:31: Thread 0 about to lock
18:01:32: Thread 1 about to release lock
18:01:32: Thread 1 after release
18:01:32: Thread 0 has lock
18:01:32: Thread 1: finishing update
18:01:32: Thread 0 about to release lock
18:01:32: Thread 0 after release
18:01:32: Thread 0: finishing update
18:01:32: Testing update. Ending value is 2


## Deadlock

In [33]:
import threading

l = threading.Lock()
print('before first acquire')
l.acquire()
print('before second acquire')
l.acquire()
print('acquired lock twice')


before first acquire
before second acquire


KeyboardInterrupt: 